In [1]:
def reverseComplement(t):
    translation={'A':'T', 'T':'A', 'G': 'C', 'C':'G'}
    revcom=''
    for i in range(len(t)):
        revcom=translation[t[i]]+revcom
    return revcom

In [2]:
def readFastq(filename):
    sequences=[]
    qualities=[]
    with open(filename,'r') as fh:
        while True:
            fh.readline()
            seq=fh.readline().rstrip()
            fh.readline()
            qual=fh.readline().rstrip()
            if len(seq)==0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities 

In [3]:
def readGenome(filename):
    genome=''
    with open(filename,'r') as fh:
        for line in fh:
            if not line[0]=='>':
                genome += line.rstrip()
    return genome
            

In [16]:
def naive_with_counts(p,t):
    hits=[]
    alignments_tried=0
    char_comparisons=0
    for i in range(len(t)-len(p)+1):
        alignments_tried+=1
        match_F=True    
        #check forward
        for j in range (len(p)):
            char_comparisons+=1
            if not t[i+j]==p[j]:
                match_F=False
                break
        if match_F: 
            hits.append(i)
    return hits, alignments_tried, char_comparisons

In [5]:
chr1=readGenome('chr1.GRCh38.excerpt.fasta') #load genome

In [17]:
p = 'word'
t = 'there would have been a time for such a word'
 

NameError: name 'A' is not defined

In [18]:
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

([40], 41, 46)


In [19]:
p = 'needle'
t = 'needle need noodle needle'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

([0, 19], 20, 35)


In [20]:
import bm_preproc

In [21]:
def boyer_moore(p,p_bm,t):
    """Do Boyer-Moore matching. p=pattern, t=text, p_bm=BoyerMoore object for p"""
    i=0
    occurrences=[]
    while i < len (t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range (len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(skip_bc, skip_gs, shift)
                mismatched=True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs=p_bm.match_skip()
            shift=max(shift,skip_gs)
        i += shift
    return occurrences
                

In [29]:
def boyer_moore_with_counts(p,p_bm,t):
    """Do Boyer-Moore matching. p=pattern, t=text, p_bm=BoyerMoore object for p"""
    i=0
    occurrences = []
    num_alignments = 0
    num_character_comparisons = 0
    while i < len (t) - len(p) + 1:
        shift = 1
        mismatched = False
        num_alignments += 1
        for j in range (len(p)-1, -1, -1):
            num_character_comparisons += 1
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(skip_bc, skip_gs, shift)
                mismatched=True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs=p_bm.match_skip()
            shift=max(shift,skip_gs)
        i += shift
    return occurrences, num_alignments, num_character_comparisons

In [31]:
p = 'word'
t = 'there would have been a time for such a word'
occurrences, num_alignments, num_character_comparisons = naive_with_counts(p, t)
print(occurrences, num_alignments, num_character_comparisons)

lowercase_alphabet = 'abcdefghijklmnopqrstuvwxyz '
p_bm = bm_preproc.BoyerMoore(p, alphabet=lowercase_alphabet)
boyer_moore_with_counts(p,p_bm,t)

([40], 41, 46)


([40], 12, 15)

In [32]:
t=chr1

In [33]:
p='GGCGCGGTGGCTCACGCCTGTAATCCCAGCACTTTGGGAGGCCGAGG'

In [34]:
p_bm = bm_preproc.BoyerMoore(p) #Q3
boyer_moore_with_counts(p,p_bm,t)

([56922], 127974, 165191)

In [35]:
naive_with_counts(p,t) #Q1, Q2

([56922], 799954, 984143)

In [144]:
import bisect


class Index(object):
    def __init__(self, t, k):
        self.k=k
        self.index=[]
        for i in range(len(t)-k+1):
            self.index.append((t[i:i+k], i))
        self.index.sort()
    
    def query(self, p):
        kmer=p[:self.k]
        #print kmer
        i=bisect.bisect_left(self.index, (kmer, -1))
        hits=[]
        #print i
        while i < len(self.index):
            if self.index[i][0] != kmer:
                
                break
            hits.append(self.index[i][1]) #appends the position of the kmer hit in the index
            i += 1
        return hits
    

In [145]:
l=[1,2,3,3,3,4,5,6]
bisect.bisect_right(l,3)

5

In [146]:
range(3)

[0, 1, 2]

In [147]:
genome=chr1
genome_index=Index(chr1, 8)

In [162]:
def pigeon_hole(p, index, genome, mm=2):
    pmer=[] #creates list n of kmers from the pattern
    n=3
    k=8
    results={}
    total_hits=0
    for i in range (n):
        pmer=p[i*k : i*k+k]
        #print pmer
    #now check if there are matches in the indext
        hits=index.query(pmer)
        total_hits+=len(hits)
        #extend if there are hits
        if len(hits)>0:
            for hit in hits:
                start=hit-(i*k) #calculates the predicted start of the match
                match=True
                if start >= 0 and start not in results.keys(): #makes sure the start is legal and a match at that start has not been found yet 
                    mismatches=0
                    for j in range(len(p)):
                        if p[j] != genome[start+j]:
                            mismatches+=1
                        if mismatches > mm:
                            match=False
                            break
                    if match:
                        results[start]=(mismatches, genome[start: start+len(p)])
    return results,total_hits
                        
                    
    


In [150]:
hits=pigeon_hole('GGCGCGGTGGCTCACGCCTGTAAT', genome_index, genome, mm=2)

In [151]:
hits

{56922: (0, 'GGCGCGGTGGCTCACGCCTGTAAT'),
 84641: (1, 'GGCGCGGTGGCTCATGCCTGTAAT'),
 147558: (1, 'GGCGCGGTGGCTCATGCCTGTAAT'),
 160162: (2, 'GGCACGGTGGCTCACGCATGTAAT'),
 160729: (1, 'GGCGCGGTGGCTCACACCTGTAAT'),
 191452: (1, 'GGCGCGGTGGTTCACGCCTGTAAT'),
 262042: (0, 'GGCGCGGTGGCTCACGCCTGTAAT'),
 273669: (1, 'GGTGCGGTGGCTCACGCCTGTAAT'),
 364263: (0, 'GGCGCGGTGGCTCACGCCTGTAAT'),
 421221: (2, 'GGCGCAATGGCTCACGCCTGTAAT'),
 429299: (1, 'AGCGCGGTGGCTCACGCCTGTAAT'),
 465647: (1, 'GGCGCAGTGGCTCACGCCTGTAAT'),
 551134: (2, 'GGGCCGGTGGCTCACGCCTGTAAT'),
 635931: (2, 'GGTGCGGTGGCTCATGCCTGTAAT'),
 657496: (0, 'GGCGCGGTGGCTCACGCCTGTAAT'),
 681737: (1, 'GGCGCGGTGGCTCATGCCTGTAAT'),
 717706: (0, 'GGCGCGGTGGCTCACGCCTGTAAT'),
 724927: (1, 'GGCACGGTGGCTCACGCCTGTAAT'),
 747359: (2, 'GGCGTGGTGGTTCACGCCTGTAAT')}

In [153]:
len(hits.keys()) #Q4

19

In [132]:
genome_index.index[1:10]

TypeError: 'list' object is not callable

In [154]:
def naive_approximate(p,t,m):
    hits=[]
    for i in range(len(t)-len(p)+1):
        match=True   
        mm=0
        #check forward
        for j in range (len(p)):
            if not t[i+j]==p[j]:
                mm+=1
                if mm > m:
                    match=False
                    break
        if match: 
            hits.append(i)
    return hits

In [157]:
len(naive_approximate('GGCGCGGTGGCTCACGCCTGTAAT', genome, 2)) #check Q4

19

In [158]:
p='GGCGCGGTGGCTCACGCCTGTAAT'

In [163]:
hits, total_hits=pigeon_hole(p, genome_index, genome, mm=2)

In [164]:
hits

{56922: (0, 'GGCGCGGTGGCTCACGCCTGTAAT'),
 84641: (1, 'GGCGCGGTGGCTCATGCCTGTAAT'),
 147558: (1, 'GGCGCGGTGGCTCATGCCTGTAAT'),
 160162: (2, 'GGCACGGTGGCTCACGCATGTAAT'),
 160729: (1, 'GGCGCGGTGGCTCACACCTGTAAT'),
 191452: (1, 'GGCGCGGTGGTTCACGCCTGTAAT'),
 262042: (0, 'GGCGCGGTGGCTCACGCCTGTAAT'),
 273669: (1, 'GGTGCGGTGGCTCACGCCTGTAAT'),
 364263: (0, 'GGCGCGGTGGCTCACGCCTGTAAT'),
 421221: (2, 'GGCGCAATGGCTCACGCCTGTAAT'),
 429299: (1, 'AGCGCGGTGGCTCACGCCTGTAAT'),
 465647: (1, 'GGCGCAGTGGCTCACGCCTGTAAT'),
 551134: (2, 'GGGCCGGTGGCTCACGCCTGTAAT'),
 635931: (2, 'GGTGCGGTGGCTCATGCCTGTAAT'),
 657496: (0, 'GGCGCGGTGGCTCACGCCTGTAAT'),
 681737: (1, 'GGCGCGGTGGCTCATGCCTGTAAT'),
 717706: (0, 'GGCGCGGTGGCTCACGCCTGTAAT'),
 724927: (1, 'GGCACGGTGGCTCACGCCTGTAAT'),
 747359: (2, 'GGCGTGGTGGTTCACGCCTGTAAT')}

In [165]:
total_hits

90

In [177]:
def naive(p,t):
    hits=[]
    for i in range(len(t)-len(p)+1):
        match_F=True    
        #check forward
        for j in range (len(p)):
            if not t[i+j]==p[j]:
                match_F=False
                break
        #check reverse
        if match_F : 
            hits.append(i)
    return hits

In [182]:
total_hits=[]
for i in range (3):
    kmer=p[i*8:i*8+8]
    total_hits=total_hits+(naive(kmer, genome))


In [184]:
total_hits

[56922,
 57056,
 83720,
 84641,
 147558,
 160729,
 191452,
 262042,
 364263,
 657496,
 681737,
 717706,
 725061,
 18878,
 56930,
 160170,
 262050,
 273677,
 282012,
 364271,
 421229,
 429307,
 465655,
 472642,
 489446,
 551142,
 621370,
 657504,
 717714,
 724935,
 18749,
 19182,
 22413,
 22548,
 23019,
 23154,
 43143,
 56938,
 67379,
 83736,
 83879,
 84657,
 84791,
 108126,
 129010,
 147574,
 160745,
 175326,
 186012,
 187671,
 191468,
 205397,
 251106,
 251240,
 262058,
 273685,
 282020,
 322751,
 364279,
 364412,
 421237,
 429315,
 454348,
 465663,
 471982,
 480517,
 480658,
 523101,
 551150,
 551843,
 572212,
 588494,
 595557,
 613475,
 621507,
 632321,
 635947,
 646504,
 651539,
 657512,
 674072,
 681753,
 707167,
 717722,
 719434,
 724943,
 746636,
 747375,
 747511,
 760505]

In [269]:
import bisect

class SubseqIndex(object):
    """Holds a subsequence index for a text T"""
    
    def __init__(self, t, k, ival):
        self.k=k
        self.ival=ival
        self.index=[]
        self.span=(k-1)*ival+1 #number of characters spanned by a kmer given ival
        for i in range (len(t)-self.span+1):
            self.index.append((t[i:i+self.span:ival], i))
        self.index.sort()
    
    def query(self, p):
        subseq=p[:self.span:self.ival]
        #print subseq
        i=bisect.bisect_left(self.index, (subseq, -1))
        hits=[]
        while i < len(self.index):
            if self.index[i][0] != subseq:
                
                break
            hits.append(self.index[i][1]) #appends the position of the kmer hit in the index
            i += 1
        return hits
        
            

In [270]:
subseq_genome_index=SubseqIndex(chr1,8,3)

In [276]:
def test_extended_match(pmer, t,mm=2):
    mismatches=0
    for i in range(len(pmer)):
        if pmer[i]!= t[i]:
            mismatches+=1
            if mismatches > mm:
                return False, 4
    return True, mismatches

In [279]:
def pigeon_hole_sub(p,genome,index, mm=2):
    k=index.k
    ival=index.ival
    span=index.span
    print 'span', span, k, ival
    pmer=[]
    hits={}
    total_hits=0
    for i in range (3):
        pmer=p[i:i+span]
        #print pmer
        index_hits=index.query(pmer)
        total_hits += len(index_hits)
        for hit in index_hits:
            #print hit
            start=hit-i
            if start >= 0 and start+span <= len(genome): #again to make sure it is a legal start!
                match, mismatches=test_extended_match(pmer, genome[start: start+len(pmer)], mm)
                if match:
                    hits[start]=(genome[start:start+len(pmer)], mismatches)
    return hits, total_hits

            
            
        
        
        

In [281]:
pigeon_hole_sub('GGCGCGGTGGCTCACGCCTGTAAT', chr1, subseq_genome_index)[1]

span 22 8 3


79

In [274]:
subseq_genome_index.query("GGCGCGGTGGCTCACGCCTGTA")

[56922,
 67486,
 83863,
 84641,
 84775,
 124024,
 147558,
 191452,
 199607,
 262042,
 262174,
 273669,
 322735,
 364263,
 421354,
 454332,
 465647,
 471966,
 472634,
 489019,
 558456,
 579737,
 596898,
 635931,
 651523,
 657496,
 658702,
 681737,
 707151,
 712449,
 717706,
 719418,
 719557,
 746620,
 747359]

In [ ]:
readFastq('/Users/venturaa/Google Drive/Computational')